In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import ChebConv  # Using Chebyshev convolution
from torch_geometric.nn import knn_graph
from torch_geometric.utils import get_laplacian
from torch_sparse import coalesce
from torch_geometric.utils import degree

In [6]:
def compute_edge_indices(point_cloud, k=6):
    """
    Compute edge indices using k-nearest neighbors for the point cloud.
    
    Parameters:
        point_cloud (torch.Tensor): Tensor of shape [N, F], where N is the number of points and F is the feature dimension (e.g., 3 for x, y, z coordinates).
        k (int): Number of nearest neighbors to consider.

    Returns:
        torch.Tensor: Edge index tensor of shape [2, num_edges].
    """
    edge_index = knn_graph(point_cloud, k=k, loop=False)
    return edge_index

# Example usage:
# point_cloud: Tensor of shape [N, 3] (N points, each with x, y, z coordinates)
# k: Number of nearest neighbors to connect each point to

In [7]:


def compute_manual_laplacian(edge_index, num_nodes):
    """
    Manually compute the Laplacian matrix without changing the shape of edge_index.
    
    Parameters:
        edge_index (torch.Tensor): Edge index tensor of shape [2, num_edges].
        num_nodes (int): Number of nodes in the point cloud.

    Returns:
        torch.Tensor: Laplacian edge weights.
    """
    # Get degrees for each node
    row, col = edge_index
    deg = degree(row, num_nodes=num_nodes, dtype=torch.float32)

    # Compute normalized Laplacian weights (L = D - A)
    edge_weight = torch.ones((edge_index.size(1), ), dtype=torch.float32)  # Weight all edges equally
    deg_inv_sqrt = deg.pow(-0.5)
    deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0  # Avoid division by zero

    # Apply normalization to the edge weights
    edge_weight = deg_inv_sqrt[row] * edge_weight * deg_inv_sqrt[col]
    
    return edge_weight



In [9]:
def train_autoencoder(model, data, edge_index, laplacian, epochs=100, lr=1e-3):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.MSELoss()

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()

        # Forward pass
        reconstructed = model(data, edge_index, laplacian)
        loss = loss_fn(reconstructed, data)

        # Backward pass
        loss.backward()
        optimizer.step()

        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')

# Example usage:
# data: point cloud data as input features (e.g., [N, F])
# edge_index: adjacency list of the point cloud or mesh (graph edges)
# laplacian: precomputed Laplacian matrix for diffusion
# 
point_cloud = torch.randn(1000, 3)  # torch.Size([1000, 3])
edge_index = compute_edge_indices(point_cloud, k=6) # torch.Size([2, 6000]) Torch.int64

num_nodes = point_cloud.size(0)
# Compute the Laplacian edge weights manually

laplacian = compute_manual_laplacian(edge_index, num_nodes) # torch.Size([6000])

# laplacian, edge_weight = get_laplacian(edge_index, normalization='sym', num_nodes=num_nodes)

in_features = 3  # x, y, z coordinates for each point
hidden_features = 64
latent_dim = 32

from models import DiffusionNetAutoencoder
model = DiffusionNetAutoencoder(in_features, hidden_features, latent_dim)

# Example training
# Replace point_cloud, edge_index, and laplacian with your actual data
train_autoencoder(model, point_cloud, edge_index, laplacian)

Epoch 0, Loss: 18.19139862060547
Epoch 10, Loss: 1.039462685585022
Epoch 20, Loss: 0.974716067314148
Epoch 30, Loss: 0.9610922932624817
Epoch 40, Loss: 0.9534638524055481
Epoch 50, Loss: 0.9489578604698181
Epoch 60, Loss: 0.9453242421150208
Epoch 70, Loss: 0.940920889377594
Epoch 80, Loss: 0.935873806476593
Epoch 90, Loss: 0.9286730289459229


In [3]:
up_teeth_nums16 = [18, 17, 16, 15, 14, 13, 12, 11, 21, 22,
                23, 24, 25, 26, 27, 28]  # Jaw_id = 2 верхняя / по 16 зубов
dw_teeth_nums16 = [38, 37, 36, 35, 34, 33, 32, 31, 41, 42,
                    43, 44, 45, 46, 47, 48]  # Jaw_id = 1 нижняя  / 16 зубов
print(dw_teeth_nums16+up_teeth_nums16)

[38, 37, 36, 35, 34, 33, 32, 31, 41, 42, 43, 44, 45, 46, 47, 48, 18, 17, 16, 15, 14, 13, 12, 11, 21, 22, 23, 24, 25, 26, 27, 28]


In [9]:
# concatenate datasets
import os, torch
from alignment_ae_dataset import AlignerDataset
ds_dir = 'datasets_align'
ds_files_list = os.listdir(ds_dir)
datasets = [torch.load(os.path.join(ds_dir, ds_file)) for ds_file in ds_files_list if not "dataset" in ds_file]

ds = torch.utils.data.ConcatDataset(datasets)
concatenated_ds_fname = 'dataset_256.pth'
torch.save(ds, os.path.join(ds_dir, concatenated_ds_fname))
print(f"{concatenated_ds_fname} saved to {ds_dir}/, length - {len(ds)}")


C:\Users\5010858\AppData\Local\Temp/ipykernel_27056/821819896.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  datasets = [torch.load(os.path.join(ds_dir, ds_file)) for d

dataset dataset_256.pth saved to datasets_align, length - 957
